In [1]:
#download the required built-in modules
import pandas as pd
#import numpy as np
from sqlalchemy import create_engine

In [2]:
#download the required csv file
df2 = pd.read_csv("./Resources/df_2.csv")
df2

,Unnamed: 0,Event,Date,Location,Disease,Death toll (estimate),Ref.
0,0,1350 BC plague of Megiddo,1350 BC,"Megiddo, land of Canaan","Amarna letters EA 244, Biridiya, mayor of Megi...",Unknown,[25]
1,1,Plague of Athens,429–426 BC,"Greece, Libya, Egypt, Ethiopia","Unknown, possibly typhus, typhoid fever or vir...","75,000–100,000",[26][27][28][29]
2,2,412 BC epidemic,412 BC,"Greece (Northern Greece, Roman Republic)","Unknown, possibly influenza",Unknown,[30]
3,3,Antonine Plague,165–180 (possibly up to 190),Roman Empire,"Unknown, possibly smallpox",5–10 million,[31][32]
4,4,Jian'an Plague,217,Han Dynasty,"Unknown, possibly typhoid fever or viral hemor...",Unknown,[33][34]
...,...,...,...,...,...,...,...
248,248,2020 Nigeria yellow fever epidemic,2020,Nigeria,Yellow fever,296 (as of 31 December 2020),[306]
249,249,2021 India black fungus epidemic,2021–present,India,Black fungus / COVID-19 associated mucormycosis,4332,[307]
250,250,2022 hepatitis of unknown origin in children,2021–present,Worldwide,Hepatitis by Adenovirus variant AF41 (Unconfir...,18,[308][309][310]
251,251,2022 monkeypox outbreak,2022–present,Worldwide,Monkeypox virus,136,[311][312][313][314]


In [3]:
#Rename columns as required
df2.rename(columns = {"Unnamed: 0":"ID"}, inplace = True)
df2

,ID,Event,Date,Location,Disease,Death toll (estimate),Ref.
0,0,1350 BC plague of Megiddo,1350 BC,"Megiddo, land of Canaan","Amarna letters EA 244, Biridiya, mayor of Megi...",Unknown,[25]
1,1,Plague of Athens,429–426 BC,"Greece, Libya, Egypt, Ethiopia","Unknown, possibly typhus, typhoid fever or vir...","75,000–100,000",[26][27][28][29]
2,2,412 BC epidemic,412 BC,"Greece (Northern Greece, Roman Republic)","Unknown, possibly influenza",Unknown,[30]
3,3,Antonine Plague,165–180 (possibly up to 190),Roman Empire,"Unknown, possibly smallpox",5–10 million,[31][32]
4,4,Jian'an Plague,217,Han Dynasty,"Unknown, possibly typhoid fever or viral hemor...",Unknown,[33][34]
...,...,...,...,...,...,...,...
248,248,2020 Nigeria yellow fever epidemic,2020,Nigeria,Yellow fever,296 (as of 31 December 2020),[306]
249,249,2021 India black fungus epidemic,2021–present,India,Black fungus / COVID-19 associated mucormycosis,4332,[307]
250,250,2022 hepatitis of unknown origin in children,2021–present,Worldwide,Hepatitis by Adenovirus variant AF41 (Unconfir...,18,[308][309][310]
251,251,2022 monkeypox outbreak,2022–present,Worldwide,Monkeypox virus,136,[311][312][313][314]


In [4]:
# Clean-up the Date column as follows:
# 1) Split the column into 2: Start Date and End Date
# 2) Create a column to capture BC and AD details
# 3) Remove the unnecessary text
# 4) Replace the value "present" with 9999. This helps with dataload as this is a Date field.

In [5]:
# Define the functions as required

# Function to capture start and end values
def get_start_end(alist):
    if len(alist) == 1:
        return alist * 2
    return alist

# Function to capture the BC and AD details
def get_bc_ad(dlist):
    if len(dlist) > 1:
        if dlist[1] != "BC":
            dlist[1] = "AD"
            return dlist
        else:
            return dlist
    else:
        dlist.append("AD")
        return dlist

In [6]:
# Split the values in Date column by hyphen
df2["Date"] = df2["Date"].str.split("–")

# Capture the start and end values using the function get_start_end
df2["Date"] = df2["Date"].apply(get_start_end)

# Store both the start and end values into a list
epidemic_date = df2["Date"].apply(pd.Series)
epidemic_date.columns = ["Start_Date", "End_Date"]
epidemic_date

,Start_Date,End_Date
0,1350 BC,1350 BC
1,429,426 BC
2,412 BC,412 BC
3,165,180 (possibly up to 190)
4,217,217
...,...,...
248,2020,2020
249,2021,present
250,2021,present
251,2022,present


In [7]:
# Capture only the year values from Start Date, removing all other text
epidemic_date["Start_Date"] = epidemic_date["Start_Date"].str.split(" ").str[0]

# Capture only the year values from End Date
epidemic_date["End_Date"] = epidemic_date["End_Date"].str.split(" ",1)

# Capture the BC and AD values, using the function get_bc_ad function defined above
epidemic_date["End_Date"] = epidemic_date["End_Date"].apply(get_bc_ad)

# Store the End date and BC/AD details in a list
epidemic_period = epidemic_date["End_Date"].apply(pd.Series)
epidemic_period.columns = ["End_Date", "BC_AD"]

# Replace the string "present" with 9999
epidemic_period["End_Date"] = epidemic_period["End_Date"].replace("present",9999)

epidemic_period

,End_Date,BC_AD
0,1350,BC
1,426,BC
2,412,BC
3,180,AD
4,217,AD
...,...,...
248,2020,AD
249,9999,AD
250,9999,AD
251,9999,AD


In [8]:
# Drop the unnecessary columns from the lists created above and merge as required
epidemic_date = epidemic_date.drop("End_Date",axis=1)

epidemic_date_cleaned = pd.concat([epidemic_date,epidemic_period],axis=1)

# Create a dataframe to capture the clean date values
df2_cleaned = pd.concat([df2, epidemic_date_cleaned],axis=1)

# Drop the original Date column
df2_cleaned = df2_cleaned.drop("Date",axis=1)
df2_cleaned

,ID,Event,Location,Disease,Death toll (estimate),Ref.,Start_Date,End_Date,BC_AD
0,0,1350 BC plague of Megiddo,"Megiddo, land of Canaan","Amarna letters EA 244, Biridiya, mayor of Megi...",Unknown,[25],1350,1350,BC
1,1,Plague of Athens,"Greece, Libya, Egypt, Ethiopia","Unknown, possibly typhus, typhoid fever or vir...","75,000–100,000",[26][27][28][29],429,426,BC
2,2,412 BC epidemic,"Greece (Northern Greece, Roman Republic)","Unknown, possibly influenza",Unknown,[30],412,412,BC
3,3,Antonine Plague,Roman Empire,"Unknown, possibly smallpox",5–10 million,[31][32],165,180,AD
4,4,Jian'an Plague,Han Dynasty,"Unknown, possibly typhoid fever or viral hemor...",Unknown,[33][34],217,217,AD
...,...,...,...,...,...,...,...,...,...
248,248,2020 Nigeria yellow fever epidemic,Nigeria,Yellow fever,296 (as of 31 December 2020),[306],2020,2020,AD
249,249,2021 India black fungus epidemic,India,Black fungus / COVID-19 associated mucormycosis,4332,[307],2021,9999,AD
250,250,2022 hepatitis of unknown origin in children,Worldwide,Hepatitis by Adenovirus variant AF41 (Unconfir...,18,[308][309][310],2021,9999,AD
251,251,2022 monkeypox outbreak,Worldwide,Monkeypox virus,136,[311][312][313][314],2022,9999,AD


In [9]:
# Clean-up the "Death toll (estimate)" column as follows:
# 1) Capture all data in this column into a new column as Comments, so the existing comments in this column aren't lost
# 2) Remove the word million and capture million in numbers (eg., 2 million captured as 2,000,000)
# 3) Reopve the commas and other formatting from the numbers
# 4) Split the column to capture the Minimum and Maximum values separately
# 5) Replace the string "Unknown" by blanks. This helps with dataload as this is an integer field.

In [10]:
# Rename the Death toll column (this is done so comments aren't lost in ETL)
df2_cleaned.rename(columns = {"Death toll (estimate)":"Comments Death toll (estimate)"}, inplace = True)
df2_cleaned

,ID,Event,Location,Disease,Comments Death toll (estimate),Ref.,Start_Date,End_Date,BC_AD
0,0,1350 BC plague of Megiddo,"Megiddo, land of Canaan","Amarna letters EA 244, Biridiya, mayor of Megi...",Unknown,[25],1350,1350,BC
1,1,Plague of Athens,"Greece, Libya, Egypt, Ethiopia","Unknown, possibly typhus, typhoid fever or vir...","75,000–100,000",[26][27][28][29],429,426,BC
2,2,412 BC epidemic,"Greece (Northern Greece, Roman Republic)","Unknown, possibly influenza",Unknown,[30],412,412,BC
3,3,Antonine Plague,Roman Empire,"Unknown, possibly smallpox",5–10 million,[31][32],165,180,AD
4,4,Jian'an Plague,Han Dynasty,"Unknown, possibly typhoid fever or viral hemor...",Unknown,[33][34],217,217,AD
...,...,...,...,...,...,...,...,...,...
248,248,2020 Nigeria yellow fever epidemic,Nigeria,Yellow fever,296 (as of 31 December 2020),[306],2020,2020,AD
249,249,2021 India black fungus epidemic,India,Black fungus / COVID-19 associated mucormycosis,4332,[307],2021,9999,AD
250,250,2022 hepatitis of unknown origin in children,Worldwide,Hepatitis by Adenovirus variant AF41 (Unconfir...,18,[308][309][310],2021,9999,AD
251,251,2022 monkeypox outbreak,Worldwide,Monkeypox virus,136,[311][312][313][314],2022,9999,AD


In [11]:
# Define the functions as required

# Function to replace the word million with actual value and to remove commas from the numbers
def replace_millions(string):
    
    # find the word million; if not pressent, remove the commas from the number and return it as such    
    try:
        million_index = string.index("million")
    except:
        return (string.replace(",",""))

    # if million is found, check if the string has one or 2 values
    else:
        try:
            # if there is a hyphen in the string, it has 2 values. so muliply both the values with 1000000 
            hyphen_index = string.index("–")
        except:
            # if there is no hyphen, then multiply this one value with 1000000
            new_string = string[:million_index]
            new_string = new_string.strip()
            try:
                new_string = int(new_string) * 1000000
            except:
                new_string = float(new_string) * 1000000
            return str(new_string)

        else:
            string1 = string.split("–",1)
            string1[0] = string1[0].strip()
            try:
                string1[0] = int(string1[0]) * 1000000
            except:
                string1[0] = float(string1[0]) * 1000000
                
            
            million_index = string1[1].index("million")
            string1[1] = string1[1][:million_index]
            string1[1] = string1[1].strip()
            try:
                string1[1] = int(string1[1]) * 1000000
            except:
                string1[1] = float(string1[1]) * 1000000
            return (str(string1[0])+"–"+str(string1[1]))

In [12]:
# use the function replace_millions to capture million in numbers
df2_cleaned["Death toll"] = df2_cleaned["Comments Death toll (estimate)"].apply(replace_millions)
df2_cleaned.head(15)

,ID,Event,Location,Disease,Comments Death toll (estimate),Ref.,Start_Date,End_Date,BC_AD,Death toll
0,0,1350 BC plague of Megiddo,"Megiddo, land of Canaan","Amarna letters EA 244, Biridiya, mayor of Megi...",Unknown,[25],1350,1350,BC,Unknown
1,1,Plague of Athens,"Greece, Libya, Egypt, Ethiopia","Unknown, possibly typhus, typhoid fever or vir...","75,000–100,000",[26][27][28][29],429,426,BC,75000–100000
2,2,412 BC epidemic,"Greece (Northern Greece, Roman Republic)","Unknown, possibly influenza",Unknown,[30],412,412,BC,Unknown
3,3,Antonine Plague,Roman Empire,"Unknown, possibly smallpox",5–10 million,[31][32],165,180,AD,5000000–10000000
4,4,Jian'an Plague,Han Dynasty,"Unknown, possibly typhoid fever or viral hemor...",Unknown,[33][34],217,217,AD,Unknown
5,5,Plague of Cyprian,Europe,"Unknown, possibly smallpox",Unknown,[35][36],250,266,AD,Unknown
6,6,Plague of Justinian (beginning of first plague...,Europe and West Asia,Bubonic plague,15–100 million,[7][37][38],541,549,AD,15000000–100000000
7,7,Roman Plague of 590 (part of first plague pand...,"Rome, Byzantine Empire",Bubonic plague,Unknown,[39],590,590,AD,Unknown
8,8,Plague of Sheroe (part of First plague pandemic),Bilad al-Sham,Bubonic plague,"25,000+",NaN,627,628,AD,25000+
9,9,Plague of Amwas (part of first plague pandemic),"Byzantine Empire, West Asia, Africa",Bubonic plague,"25,000+",[40],638,639,AD,25000+


In [13]:
# Split the column to capture the Minimum and Maximum values separately
df2_cleaned["Death toll"] = df2_cleaned["Death toll"].str.split("–",1)

# Capture the min and max values using the function get_start_end
df2_cleaned["Death toll"] = df2_cleaned["Death toll"].apply(get_start_end)

# Store both the min and max values into a list
death_toll = df2_cleaned["Death toll"].apply(pd.Series)
death_toll.columns = ["Min_Death", "Max_Death"]
death_toll.head(15)

,Min_Death,Max_Death
0,Unknown,Unknown
1,75000,100000
2,Unknown,Unknown
3,5000000,10000000
4,Unknown,Unknown
5,Unknown,Unknown
6,15000000,100000000
7,Unknown,Unknown
8,25000+,25000+
9,25000+,25000+


In [14]:
# Remove the unneccesary text from the data and replace "Unknown" with null
death_toll["Min_Death_Estimate"]=(death_toll["Min_Death"].str.extract('(\d+)')
                                                         .astype('Int64'))

death_toll["Max_Death_Estimate"]=(death_toll["Max_Death"].str.extract('(\d+)')
                                                         .astype('Int64'))
death_toll

,Min_Death,Max_Death,Min_Death_Estimate,Max_Death_Estimate
0,Unknown,Unknown,<NA>,<NA>
1,75000,100000,75000,100000
2,Unknown,Unknown,<NA>,<NA>
3,5000000,10000000,5000000,10000000
4,Unknown,Unknown,<NA>,<NA>
...,...,...,...,...
248,296 (as of 31 December 2020),296 (as of 31 December 2020),296,296
249,4332,4332,4332,4332
250,18,18,18,18
251,136,136,136,136


In [15]:
# merge the cleaned dataframe to the main dataframe
events_details = pd.concat([df2_cleaned, death_toll],axis=1)

# Drop the unnecessary columns
events_details = events_details.drop(columns=["Death toll","Min_Death","Max_Death"],axis=1)

events_details

,ID,Event,Location,Disease,Comments Death toll (estimate),Ref.,Start_Date,End_Date,BC_AD,Min_Death_Estimate,Max_Death_Estimate
0,0,1350 BC plague of Megiddo,"Megiddo, land of Canaan","Amarna letters EA 244, Biridiya, mayor of Megi...",Unknown,[25],1350,1350,BC,<NA>,<NA>
1,1,Plague of Athens,"Greece, Libya, Egypt, Ethiopia","Unknown, possibly typhus, typhoid fever or vir...","75,000–100,000",[26][27][28][29],429,426,BC,75000,100000
2,2,412 BC epidemic,"Greece (Northern Greece, Roman Republic)","Unknown, possibly influenza",Unknown,[30],412,412,BC,<NA>,<NA>
3,3,Antonine Plague,Roman Empire,"Unknown, possibly smallpox",5–10 million,[31][32],165,180,AD,5000000,10000000
4,4,Jian'an Plague,Han Dynasty,"Unknown, possibly typhoid fever or viral hemor...",Unknown,[33][34],217,217,AD,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...
248,248,2020 Nigeria yellow fever epidemic,Nigeria,Yellow fever,296 (as of 31 December 2020),[306],2020,2020,AD,296,296
249,249,2021 India black fungus epidemic,India,Black fungus / COVID-19 associated mucormycosis,4332,[307],2021,9999,AD,4332,4332
250,250,2022 hepatitis of unknown origin in children,Worldwide,Hepatitis by Adenovirus variant AF41 (Unconfir...,18,[308][309][310],2021,9999,AD,18,18
251,251,2022 monkeypox outbreak,Worldwide,Monkeypox virus,136,[311][312][313][314],2022,9999,AD,136,136
